In [11]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict

In [12]:
# create state
class RunsAnalysis(TypedDict):
    runs: int
    balls: int
    fours: int
    sixes: int

    sr: float
    bpb: float
    boundary_percentage: float

    summary: str

In [17]:
def calculate_sr(state: RunsAnalysis) -> RunsAnalysis:
    # extract runs and balls
    runs = state['runs']
    balls = state['balls']

    # calculate strike rate
    sr = (runs / balls)*100

    # update strike rate and returning partial state
    return {'sr': sr}
    

In [18]:
def calculate_bpb(state: RunsAnalysis) -> RunsAnalysis:
    # extract balls, fours, sixes
    balls = state['balls']
    boundaries = state['fours'] + state['sixes']

    # calculate balls per boundary
    bpb = balls / boundaries

    # update bpb of state and return partial state
    return {'bpb': bpb}

In [19]:
def calculate_boundary_percentage(state: RunsAnalysis) -> RunsAnalysis:
    # extract runs and boundaries
    runs = state['runs']
    boundaries = (state['fours'] * 4) + (state['sixes'] * 6)

    # calculate boundary percentage
    boundary_percentage = (boundaries / runs) * 100

    # update boundary percentage of state and returning partial state
    return {'boundary_percentage': boundary_percentage}

In [20]:
def calculate_summary(state: RunsAnalysis) -> RunsAnalysis:
    # create summary
    summary = f"""
    Runs scored - {state['runs']} \n
    Strike rate - {state['sr']} \n
    Ball per boundary - {state['bpb']} \n
    Boundary percentage - {state['boundary_percentage']}
"""
    # update the state and returning the partial state
    return {'summary': summary}

In [22]:
# create graph
graph = StateGraph(RunsAnalysis)

# add nodes
graph.add_node('calculate_sr', calculate_sr)
graph.add_node('calculate_bpb', calculate_bpb)
graph.add_node('calculate_boundary_percentage', calculate_boundary_percentage)
graph.add_node('calculate_summary', calculate_summary)

# add edges
graph.add_edge(START, 'calculate_sr')
graph.add_edge(START, 'calculate_bpb')
graph.add_edge(START, 'calculate_boundary_percentage')

graph.add_edge('calculate_sr', 'calculate_summary')
graph.add_edge('calculate_bpb', 'calculate_summary')
graph.add_edge('calculate_boundary_percentage', 'calculate_summary')

graph.add_edge('calculate_summary', END)


# compile graph
workflow = graph.compile()

In [25]:
initial_state = {'runs': 100, 'balls': 50, 'fours': 6, 'sixes': 4}
final_state = workflow.invoke(initial_state)

print(final_state['summary'])


    Runs scored - 100 

    Strike rate - 200.0 

    Ball per boundary - 5.0 

    Boundary percentage - 48.0

